In [1]:

# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

# get_ipython().run_line_magic('matplotlib', 'inline')


# In[2]:



import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold

from sklearn import metrics
import json

import warnings
warnings.filterwarnings("ignore")

sys.path.append('..')
from lib.line_notif import send_message
from lib.utils import reduce_mem_usage, current_time, unpickle, to_pickle
from lib.utils import one_hot_encoder, apply_agg, multi_combine_categorical_feature
from lib.utils import import_data, get_split_indexer 

In [5]:
df_list = []
for f in np.sort(glob("../processed/v003/coulomb_feat/coulomb_train_*.pkl")):
    coulomb_test = np.load(f)
    df_list += [coulomb_test]

In [6]:
coulomb_test_df = pd.concat(df_list, axis=0)

In [7]:
coulomb_test_df.shape

(756113, 27)

In [8]:

df_test = pd.read_csv('../input/test.csv')

In [12]:
df_test.shape

(2505542, 5)

In [9]:
df_test.molecule_name.nunique()

45772

In [10]:
coulomb_test_df.head()

,molecule_name,atom_index,dist_H_0,dist_H_1,dist_H_2,dist_H_3,dist_H_4,dist_C_0,dist_C_1,dist_C_2,dist_C_3,dist_C_4,dist_N_0,dist_N_1,dist_N_2,dist_N_3,dist_N_4,dist_O_0,dist_O_1,dist_O_2,dist_O_3,dist_O_4,dist_F_0,dist_F_1,dist_F_2,dist_F_3,dist_F_4
0,dsgdb9nsd_000004,0,0.88648,0.19558,0.00000,0.00000,0.00000,0.69551,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,dsgdb9nsd_000004,1,0.88648,0.19558,0.00000,0.00000,0.00000,0.69551,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,dsgdb9nsd_000004,2,0.09055,0.00000,0.00000,0.00000,0.00000,0.88648,0.19558,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,dsgdb9nsd_000004,3,0.09055,0.00000,0.00000,0.00000,0.00000,0.88648,0.19558,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,dsgdb9nsd_000015,0,0.83729,0.82296,0.82296,0.14348,0.14345,0.18300,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.50580,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [11]:
coulomb_test_df.molecule_name.nunique()

45772

In [14]:
df_test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [13]:
import time
def merge_atom(df, df_distance):
    df_merge_0 = pd.merge(df, df_distance, left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name', 'atom_index'])
    df_merge_0_1 = pd.merge(df_merge_0, df_distance, left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name', 'atom_index'])
    del df_merge_0_1['atom_index_x'], df_merge_0_1['atom_index_y']
    return df_merge_0_1

In [ ]:
start = time.time()
df_test_dist = merge_atom(df_test, coulomb_test_df).sort_values("id")
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

drop_cols = ["molecule_name","atom_index_0","atom_index_1","type"]
df_test_dist.drop(drop_cols, axis=1).to_csv("../processed/v003/coulomb_interaction_test.csv")

In [19]:
df_test_dist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2505542 entries, 0 to 2505536
Data columns (total 55 columns):
id               int64
molecule_name    object
atom_index_0     int64
atom_index_1     int64
type             object
dist_H_0_x       float64
dist_H_1_x       float64
dist_H_2_x       float64
dist_H_3_x       float64
dist_H_4_x       float64
dist_C_0_x       float64
dist_C_1_x       float64
dist_C_2_x       float64
dist_C_3_x       float64
dist_C_4_x       float64
dist_N_0_x       float64
dist_N_1_x       float64
dist_N_2_x       float64
dist_N_3_x       float64
dist_N_4_x       float64
dist_O_0_x       float64
dist_O_1_x       float64
dist_O_2_x       float64
dist_O_3_x       float64
dist_O_4_x       float64
dist_F_0_x       float64
dist_F_1_x       float64
dist_F_2_x       float64
dist_F_3_x       float64
dist_F_4_x       float64
dist_H_0_y       float64
dist_H_1_y       float64
dist_H_2_y       float64
dist_H_3_y       float64
dist_H_4_y       float64
dist_C_0_y       flo